## 정형 데이터와 비정형 데이터
생선 - 길이, 높이, 무게 등으로 이루어진 데이터

와인 - 알코올 도수, 산도, 당도 등으로 이루어진 데이터

위와 같이 csv 파일에 보기 좋게 정리할 수 있는 데이터를 **정형 데이터(structured data)**라고 하고, 텍스트 데이터, 사진, 음악 등의 csv 파일로 정리하기 어려운 데이터를 **비정형 데이터(unstructured data)**라고 한다. 정형 데이터를 예측하는 데 가장 뛰어난 성능을 보이는 머신러닝 기법이 **앙상블 학습(ensemble learning)**이다. 앙상블 학습이란 여러 개의 약 분류기(weak classifier)를 합쳐 하나의 강 분류기(strong classifier)를 만드는 것인데, 여러 개의 결정 트리를 결합하여 하나의 결정 트리보다 더 높은 성능을 내는 기법이다. 앙상블 학습의 예시로는 랜덤 포레스트, 엑스트라 트리, 그레이디언트 부스팅, 히스토그램 기반 그레이디언트 부스팅 등이 있다

## 랜덤 포레스트
이름 그대로 여러 결정트리를 랜덤하게 만들어 각 결정 트리의 예측을 사용해 최종 예측을 만든다. '랜덤하게 만든다'에는 데이터를 랜덤하게 만든다는 의미가 포함되어있다. 예를 들어 1000개의 데이터가 있으면, 이 데이터 중에서 중복조합으로 1000개의 새로운 데이터를 만든다. 즉 1000개의 본 데이터에서 하나를 선택한 뒤 그 하나를 다시 본 데이터에 넣고, 그 다음 하나를 선택하고 다시 본 데이터에 넣는 것을 반복하여 1000개의 새로운 데이터셋을 만든다는 의미이다. 이렇게 만들어진 샘플을 **부트스트랩 샘플(bootstrap sample)**이라 부른다.

또한 각 노드를 분할할 때 전체 특성 중 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾는데, 사이킷런에서 제공하는 RandomForestClassifier에서는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택한다. 예를 들어 4개의 특성이 있다면 무작위로 2개의 특성만 골라 분할에 사용한다

사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 훈련하여 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼는다. 회귀일 때는 단순하게 각 트리의 예측을 평균한다

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 데이터 불러오기
wine = pd.read_csv('https://bit.ly/wine_csv_data')

# 타깃 데이터 분리
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine[['class']].to_numpy()

# 훈련, 테스트셋 분리
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

# 교차 검증
# return_train_score: 검증 점수뿐만 아니라 훈련셋에 대한 점수도 함께 반환
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, 
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
print()

# 특성 중요도
rf.fit(train_input, train_target)
print(rf.feature_importances_)

# 부트스트랩 샘플을 뽑을 때 운 나쁘게 선택되지 않은 샘플이 있는데, 이 샘플을
# OOB(out of bag) 샘플이라 하고 이 샘플을 검증셋처럼 사용할 수 있다
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.9973541965122431 0.8905151032797809



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


[0.23167441 0.50039841 0.26792718]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


0.8934000384837406
